This notebook will create a permanent transient topic based on candidates selected through the lens archive study.

In [ ]:
import os, requests
import pandas as pd
from ampel_notebook_utils import api_name2candids
from ampel_notebook_utils import api_name2candid
import matplotlib.pyplot as plt

In [ ]:
# Path to subselected transients
df_select = pd.read_csv('/home/jnordin/Downloads/new_candidates_SNor_gal_absmag19.5_z0.1_det5_dist3.csv')


In [ ]:
sne = list(df_select['ZTF name'])

In [ ]:
print('We will create a topic with {} supernovae'.format(len(sne)))

A AMPEL archive access token needs to be supplied to access the archive. This can be obtained from the archive tab of 
https://ampel.zeuthen.desy.de/live/dashboard/tokens
(assumes user to have joined the AmpelProject github organization). 

In [ ]:
token = os.environ["ARCHIVE_TOKEN"]   # Token here stored as system variable

In [ ]:
# Find alert candidate IDS for the list of sne. This can be done in two ways depending on
# whether you wish to include _all_ alerts distributed for a transient or only the _latest.
# A. All alerts
# Takes a while since this queries the archive for every alert for every sn
#candids = [api_name2candids(sn, token) for sn in sne]
#mycandids = [a for sncandids in candids for a in sncandids]
# B. Only the final (which includes the previous 30 day history)
mycandids = [api_name2candid(sn, token) for sn in sne]


In [ ]:
print('Preparing to create a topic containing {} alerts from {} sne'.format(len(mycandids), len(candids)))

Post topics based on this list. Start by one containing the most populous alert from each.

In [ ]:
topic_disc = 'A list of 121 alerts from 121 SNe selected as potential lens targets. Created June 7 2022.'

In [ ]:
body = {
    "description" : topic_disc,
    "candids" : mycandids
}

In [ ]:
base_url = 'https://ampel.zeuthen.desy.de/api/ztf/archive/v3'

In [ ]:
response = requests.post(f"{base_url}/topics/", headers={"Authorization": "bearer "+token}, json=body )

In [ ]:
topic_token = response.json()

In [ ]:
print(topic_token)

This is now a _permanent_ token which should be appropriately stored. Note that it is not possible retrieve a token once created through the api, hence you need to keep track of it.

*** 
_topic_:  F7nTWtgYG7PV38H5W2fFRJlHM0f-15DOIr0R55Pemsg

_description_: A list of 121 alerts from 121 SNe selected as potential lens targets. Created June 7 2022.
***



Once a topic has been defined, it can be traversed using a an alert token. Note that this needs to be recreated - it expires after going through the topic.

In [ ]:
body = {
    "topic": topic_token, 
    "chunk_size": 100
}

In [ ]:
response = requests.post(f"{base_url}/streams/from_topic", headers={"Authorization": "bearer "+token}, json=body )

In [ ]:
if not response.ok:
    print('Accessing stream from topic not successful')
    print(response)

In [ ]:
resume_token = response.json()['resume_token']

In [ ]:
# Alerts are now being queued in the archive, and can be retrieved through this token
resume_token